<a href="https://colab.research.google.com/github/cincinnatilibrary/collection-analysis/blob/master/reports/leased_items_checkouts_by_branch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHPL - Collection Analysis - **Leased Items**

<img src="https://ilsweb.cincinnatilibrary.org/img/CHPL_Brandmark_Primary.png" alt="CHPL" title="CHPL" width="300"/>

Data sources

* https://collection-analysis.cincy.pl/

In [1]:
#@title
import pandas as pd
import numpy as np
import altair as alt
# if we want to save output as .png use this
# NOTE: use !pip install -U altair_saver above to download / install first
# from altair_saver import save
from urllib.parse import urlencode
import requests

base_url = 'https://collection-analysis.cincy.pl/current_collection'

In [2]:
#@title
sql = """\
-- find the ciculation of leased items by stat group of branch
select
  strftime('%Y-%m', transaction_day) as circ_month,
  stat_group_branch_name,
  -- bib_record_num,
  count(*) as count_checkouts
from
  circ_leased_items
where
  op_code = 'o'
group by
  1,
  2
order by
  1,
  2
"""

df = pd.read_json(
    requests.get(
      url=base_url + '.json',
      params={
        'sql': sql,
        '_shape': 'array'
      }
    ).content
)

# df.head()

## Find Aggregated Check-out Circulations For Leased Items by Branch and Month 

(previous 180 days)

---



In [3]:
#@title
print('Click the link below to view the raw data:', base_url + '?' + urlencode({'sql':sql}), sep='\n')

Click the link below to view the raw data:
https://collection-analysis.cincy.pl/current_collection?sql=--+find+the+ciculation+of+leased+items+by+stat+group+of+branch%0Aselect%0A++strftime%28%27%25Y-%25m%27%2C+transaction_day%29+as+circ_month%2C%0A++stat_group_branch_name%2C%0A++--+bib_record_num%2C%0A++count%28%2A%29+as+count_checkouts%0Afrom%0A++circ_leased_items%0Awhere%0A++op_code+%3D+%27o%27%0Agroup+by%0A++1%2C%0A++2%0Aorder+by%0A++1%2C%0A++2%0A


In [4]:
#@title
# color="#0090bd"
chart = alt.Chart(df).mark_bar().encode(
    # x='end_date:T',
    x=alt.X(
        'circ_month',
        axis=alt.Axis(title='circulation month', labelAngle=-70),
        # sort='-y'
    ),
    y=alt.Y(
        'count_checkouts:Q',
        title = 'count total leased item checkouts',
        # scale = alt.Scale(domain=[0,100]),
    ),
    color=alt.Color(
        'stat_group_branch_name',
        legend=None
    ),
    tooltip=['stat_group_branch_name', 'circ_month', 'count_checkouts']
).properties(
    title="",
    # .format(df.iloc[0]['branch_name']),
    width=250,
).facet(
    facet='stat_group_branch_name',
    columns=5
)

chart.display()

alt.FacetChart(...)

## Find Percentage of Leased Items Checked-out per Location

(current day)

In [21]:
#@title
# find the percent of leased items checked-out by location 

sql = """\
with item_data as (
  select
    location_name,
    branch_name,
    item_format,
    -- item_status_code,
    -- item_status_name,
    count(
      case
        when checkout_date is not null then 1
        else null
      end
    ) as count_checkout,
    count(*) as count_total
  from
    item_view
  where
    "barcode" >= :p0
    and "barcode" < :p1
    and item_status_code not in ('m', 'n', '$', 'w', 'z')
  group by
    1,
    2,
    3 --,
    -- 4,
    -- 5
)
select
  *,
  round(
    ((count_checkout * 1.0) / (count_total * 1.0)) * 100.0,
    2
  ) as percent_checked_out
from
  item_data
"""

df = pd.read_json(
    requests.get(
      url=base_url + '.json',
      params={
        'sql': sql,
        'p0': 'L000000000000',
        'p1': 'M',
        '_shape': 'array'
      }
    ).content
)

df

,location_name,branch_name,item_format,count_checkout,count_total,percent_checked_out
0,Anderson Lucky Day Collection,Anderson,Leased Book,164,210,78.10
1,Avondale Lucky Day Collection,Avondale,Leased Book,2,15,13.33
2,Avondale Lucky Day Collection,Avondale,Leased DVD,26,60,43.33
3,Blue Ash Lucky Day Collection,Blue Ash,Leased Book,144,182,79.12
4,Bond Hill Lucky Day Collection,Bond Hill,Leased Book,6,15,40.00
5,Cheviot Lucky Day Collection,Cheviot,Leased Book,15,23,65.22
6,Cheviot Lucky Day Collection,Cheviot,Leased DVD,17,46,36.96
7,Clifton Lucky Day Collection,Clifton,Leased Book,19,45,42.22
8,College Hill Lucky Day Collection,College Hill,Leased Book,13,33,39.39
9,Corryville Lucky Day Collection,Corryville,Leased Book,4,11,36.36


In [25]:
#@title
chart = alt.Chart(df).mark_bar(color="#0090bd", opacity=.70).encode(
    # x='end_date:T',
    y=alt.Y(
        'location_name',
        axis=alt.Axis(title='location name'), # labelAngle=-90
        # sort='-x'
    ),
    x=alt.X(
        'percent_checked_out',
        axis=alt.Axis(title='percent checked-out', labelAngle=-45),
        title = 'Percent of Total Checked Out',
        scale = alt.Scale(domain=[0, 100]),
        stack=None,
        # sort='-y'
    ),
    color='item_format'
    # tooltip=['branch_name', 'item_format', 'percent_checked_out', 'count_items', 'count_current_checkout', 'sum_checkout_total', 'sum_renewal_total', 'sum_price']
).properties(
    title="Percent Checked Out Leased Items By Branch",
    width=900,
)

chart.display()

# # we can save this file to an html file: name + date if that's what we want
# chart.save(
#     'output-{}.html'.format(
#         pd.Timestamp('now').strftime('%Y-%m-%d')
#     )
# )

# # remember, if we want to use this, import altair_saver (and requirements) above
# save(chart, "chart.png")

alt.Chart(...)

In [7]:
#@title
sql = """\
select
  l.bib_record_num,
  -- l.itype_code_num,
  p.itype_name as item_format,
  cast(l.bib_record_num as TEXT) || ' -- ' || case
    when length(bib.best_title) > 30 then substr(bib.best_title, 1, 30) || ' ... '
    else bib.best_title
  end || coalesce(
    ' / ' || nullif(
      case
        when length(bib.best_author) > 20 then substr(bib.best_author, 1, 20) || ' ... '
        else bib.best_author
      end,
      ''
    ),
    ''
  ) || coalesce(' (' || bib.publish_year || ')', '') as title,
  strftime('%Y-%m', transaction_day) as circ_month,
  count(*) as count_checkouts
from
  circ_leased_items as l
  join bib on bib.bib_record_num = l.bib_record_num
  join itype_property as p on p.itype_code = l.itype_code_num
where
  op_code = 'o'
group by
  1,
  2,
  3,
  4
order by
  1 DESC,
  2 ASC
"""

df = pd.read_json(
    requests.get(
      url=base_url + '.json',
      params={
        'sql': sql,
        '_shape': 'array'
      }
    ).content
)

# df.head()

## Find Aggregated Check-out Circulations For Leased Items by Title and Month 

(previous 180 days)

**Note**: if this is difficult to view in the pane below, expand the file menu ( image.png icon to the left) 

<--

and click to download the `output.html` file



---

In [8]:
#@title
print('Click the link below to view the raw data:', base_url + '?' + urlencode({'sql':sql}), sep='\n')

Click the link below to view the raw data:
https://collection-analysis.cincy.pl/current_collection?sql=select%0A++l.bib_record_num%2C%0A++--+l.itype_code_num%2C%0A++p.itype_name+as+item_format%2C%0A++cast%28l.bib_record_num+as+TEXT%29+%7C%7C+%27+--+%27+%7C%7C+case%0A++++when+length%28bib.best_title%29+%3E+30+then+substr%28bib.best_title%2C+1%2C+30%29+%7C%7C+%27+...+%27%0A++++else+bib.best_title%0A++end+%7C%7C+coalesce%28%0A++++%27+%2F+%27+%7C%7C+nullif%28%0A++++++case%0A++++++++when+length%28bib.best_author%29+%3E+20+then+substr%28bib.best_author%2C+1%2C+20%29+%7C%7C+%27+...+%27%0A++++++++else+bib.best_author%0A++++++end%2C%0A++++++%27%27%0A++++%29%2C%0A++++%27%27%0A++%29+%7C%7C+coalesce%28%27+%28%27+%7C%7C+bib.publish_year+%7C%7C+%27%29%27%2C+%27%27%29+as+title%2C%0A++strftime%28%27%25Y-%25m%27%2C+transaction_day%29+as+circ_month%2C%0A++count%28%2A%29+as+count_checkouts%0Afrom%0A++circ_leased_items+as+l%0A++join+bib+on+bib.bib_record_num+%3D+l.bib_record_num%0A++join+itype_property+as

In [9]:
#@title
# color="#0090bd"
chart = alt.Chart(df).mark_bar().encode(
    # x='end_date:T',
    x=alt.X(
        'circ_month',
        axis=alt.Axis(title='circulation month', labelAngle=-70),
        # sort=manual_ordered
    ),
    y=alt.Y(
        'count_checkouts:Q',
        title = 'count total leased item checkouts',
        # scale = alt.Scale(domain=[0,100]),
    ),
    color=alt.Color(
        'item_format',
        # legend=None
    ),
    tooltip=['item_format', 'bib_record_num', 'title', 'circ_month', 'count_checkouts']
).properties(
    title="",
    # .format(df.iloc[0]['branch_name']),
    width=400,
).facet(
    facet='title',
    columns=3
)

chart.save('output.html')

from IPython.display import HTML
HTML('output.html')